In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests tika

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 國立臺灣大學美術史研究集刊
http://ejournal.press.ntu.edu.tw/query.php?Action_From=level&lvbw=F111
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
[3] Selenium give file name when downloading
https://stackoverflow.com/questions/34548041/selenium-give-file-name-when-downloading
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 高階文件操作工具
import shutil

# 檔案剖析工具
from tika import parser

In [ ]:
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument('--disable-gpu')
my_options.add_argument('--disable-software-rasterizer')
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"')

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'http://ejournal.press.ntu.edu.tw'
url = prefix_url + '/query.php?Action_From=level&lvbw=F111'

# 指定 sheet name
folderName = sheetName = 'ejournal_press_ntu_edu_tw_F111'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
# 設定 Chrome 下載路徑 (需要絕對路徑)
fullDownloadPath = os.getcwd() + '\\' + folderName

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": fullDownloadPath,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing_for_trusted_sources_enabled": False,
    "safebrowsing.enabled": False,
    "plugins.always_open_pdf_externally": True
})

# 放置爬取的資料
listData = []

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
# 走訪頁面
def visit():
    global driver
    
    try:
        # 走訪首頁
        driver.get(url)

        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'input[type=submit][value=中文]')
            )
        )
        
        # 點按首頁
        driver.find_element(By.CSS_SELECTOR, 'input[type=submit][value=中文]').click()
        
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.Login_Data a[href*=logout]')
            )
        )
        
        # 轉向至實際走訪頁面
        driver.get(url)
        
        # 等待
        sleep(randint(1,2))
        
        # 按下設定分頁數的連結
        driver.find_element(By.CSS_SELECTOR, 'span.control_select_off > a[onclick*="page"]').click()
    except TimeoutException as e:
        print('等待逾時: visit')
    
# 剖析元素資料
def parse():
    global driver
    
    try:
        # 刷新頁面
        driver.refresh()
        
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'select[onchange*=pageJump]')
            )
        )
        
        # 計算 select 底下的 option 數量
        numPages = len(driver.find_elements(By.CSS_SELECTOR, 'select[onchange*=pageJump]')[0].find_elements(By.CSS_SELECTOR, 'option'))
        
        # 切換分頁用的變數 (zero-based)
        idx = 0
        
        # 走訪分頁
        while idx < numPages:
            # 取得 select
            sel_elm = Select(driver.find_elements(By.CSS_SELECTOR, 'select[onchange*=pageJump]')[0])
            
            # 切換分頁
            sel_elm.select_by_index(idx)
            
            # 等待
            sleep(randint(1,2))
            
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div.DataList')
                )
            )
            
            # 過濾刊名與出版日期
            regexOrigin = r'(.+)（(\d{4}年\d{1,2}月)）'
            
            # 取得每一個
            for div in driver.find_elements(By.CSS_SELECTOR, 'div.DataList'):
                # 取得底下所有 div
                div_elms = div.find_elements(By.CSS_SELECTOR, 'div')
                
                # 篇名(中文)
                a_cht_title_elm = div_elms[0].find_element(By.CSS_SELECTOR, 'div.title_area span.data_ctitle a[href*=record]')
                strJournalChtName = a_cht_title_elm.get_attribute('innerText').strip()
                
                # 篇名(英文)
                a_eng_title_elm = div_elms[0].find_element(By.CSS_SELECTOR, 'div.title_area span.data_etitle a[href*=record]')
                strJournalEngName = a_eng_title_elm.get_attribute('innerText').strip()
                
                # 取得 出處(刊名)、作者 等元素
                div_field_value_elms = div_elms[0].find_elements(By.CSS_SELECTOR, 'div.class_area div.field_value')
                
                # 刊名、出版日期
                strOrigin = div_field_value_elms[0].get_attribute('innerText').strip()
                strOrigin = re.sub(r'\/|\n', ' ', strOrigin)
                matchOrigin = re.search(regexOrigin, strOrigin)
                strJournalTitle = matchOrigin[1]
                strPublishDate = matchOrigin[2]
             
                # 作者
                strAuthor = div_field_value_elms[1].get_attribute('innerText').strip()
                
                # 連結
                strLink = a_cht_title_elm.get_attribute('href')
                
                # 整理資料
                listData.append({
                    'id': md5(strJournalTitle + strPublishDate + strJournalChtName + strJournalEngName + strAuthor),
                    'journal_title': strJournalTitle,
                    'publish_date': strPublishDate,
                    'journal_name_cht': strJournalChtName,
                    'journal_name_eng': strJournalEngName,
                    'author': strAuthor,
                    'link': strLink
                })
                
            
            # 等待
            sleep(1)
            
            # 累計下個分頁數
            idx += 1
        
    except TimeoutException as e:
        print('等待逾時: parse')
    
# 取得 pdf
def getPdf():
    global driver, listData
    
    try:        
        # 回到首頁
        driver.get(url)
        
        # 等待
        sleep(1)
        
        # 走訪每一個內頁
        for index, myDict in enumerate(listData):
            # 進入內頁
            driver.get(myDict['link'])
            
            # 有 pdf viewer 的話，則進行 pdf 下載流程
            if len(driver.find_elements(By.CSS_SELECTOR, 'div#PDF_View object')) > 0:
                # 取得 pdf viewer 的連結
                object_elm = driver.find_element(By.CSS_SELECTOR, 'div#PDF_View object')
                strPdfLink = object_elm.get_attribute('data')

                # 開 tab 下載 pdf
                driver.execute_script(f'window.open("{strPdfLink}", "_blank");')

                # 回到原來的 tab
                driver.switch_to.window(driver.window_handles[0])

                # 等檔案下載一下，讓下載路徑至少有 .crdownload
                sleep(5)

                # 檔案下載完後，再改檔名
                while True:
                    # 取得最後下載的檔案名稱(含絕對路徑)
                    filename = max([os.path.join(fullDownloadPath, f) for f in os.listdir(fullDownloadPath)], key=lambda f : os.path.getmtime(os.path.join(fullDownloadPath, f)))
                    
                    # 若檔案還是 .part，則繼續等待
                    if 'crdownload' in filename:
                        sleep(5)
                    else:
                        # 修改檔名
                        shutil.move(filename, os.path.join(fullDownloadPath, f'{myDict["id"]}.pdf'))
                        sleep(5)
                        break
                        
    except TimeoutException as e:
        print('等待逾時: getPdf')
        
# 關閉瀏覽器
def close():
    global driver
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料，看看是否已經存在於資料表當中
        sql_query = f'''
        SELECT 1
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, journal_title, publish_date, journal_name_cht, journal_name_eng, 
            author, link, pdf_link, is_downloaded, created_at, 
            updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?,?,
            ?
        )
        '''
        
        # 放置準備寫入的資料
        list_insert = []
        
        # 將 json 資料一筆一筆找出來
        for myDict in listJson:
            # 如果資料庫沒有這筆資料(透過 id)，則將資料以 tuple 格式放到 list 當中，方便新增 bulk 資料
            if cursor.execute(sql_query, (myDict["id"],)).fetchone() == None:
                # 整合所有需要寫入的資料
                list_insert.append((
                    myDict['id'],
                    myDict['journal_title'],
                    myDict['publish_date'],
                    myDict['journal_name_cht'],
                    myDict['journal_name_eng'],
                    myDict['author'],
                    myDict['link'],
                    myDict['link'],
                    0,
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                ))
        
        # 新增資料到資料庫當中
        cursor.executemany(sql_insert, list_insert)
        
        # 執行 SQL 語法
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()
    
# 確認 pdf 檔案狀況
def checkPdf():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 將查詢出來的結果 (tuple)，變成 key-value 型式 (dict)
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢尚未確認的資料
        sql_query = f'''
        SELECT sn, id, pdf_link
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(是否下載過)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
            
        # 取得所有 is_downloaded = 0 的資料
        for myDict in cursor.execute(sql_query).fetchall():
            # 開啟 pdf 檔案
            parsed_pdf = parser.from_file(f'{folderPath}/{myDict["id"]}.pdf')

            # 若 pdf 可以開啟，則代表檔案可用
            if parsed_pdf['content'] != None:
                # 將 is_downloaded 改成 1，代表下載檔案可用
                cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()

In [ ]:
# 初始化 Web Driver
init()

In [ ]:
# 走訪頁面
visit()

In [ ]:
# 剖析元素資料
parse()

In [ ]:
# 取得 pdf (總共執行了 2515.2226753234863 秒)
time_begin = time.time()
getPdf()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 儲存成 json
saveJson()

In [ ]:
# 儲存 .db
saveDB()

In [ ]:
# 確認 pdf 檔案狀況 (總共執行了 91.64600825309753 秒)
time_begin = time.time()
checkPdf()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")